In [ ]:
%matplotlib inline


Image-based optimization with image pyramid
===========================================



We start this example by importing everything we need and setting the device we will
be working on.



In [ ]:
import pystiche
from pystiche.demo import demo_images
from pystiche.enc import vgg19_multi_layer_encoder
from pystiche.image import show_image, write_image
from pystiche.loss import PerceptualLoss
from pystiche.misc import get_device, get_input_image
from pystiche.ops import (
    FeatureReconstructionOperator,
    GramOperator,
    MultiLayerEncodingOperator,
)
from pystiche.optim import default_image_pyramid_optim_loop
from pystiche.pyramid import ImagePyramid

print(f"I'm working with pystiche=={pystiche.__version__}")

device = get_device()
print(f"I'm working with {device}")

images = demo_images()

At first we define a :class:`~pystiche.loss.perceptual.PerceptualLoss` that is used
as optimization ``criterion``.



In [ ]:
multi_layer_encoder = vgg19_multi_layer_encoder()


content_layer = "relu4_2"
content_encoder = multi_layer_encoder.extract_single_layer_encoder(content_layer)
content_weight = 1e0
content_loss = FeatureReconstructionOperator(
    content_encoder, score_weight=content_weight
)


style_layers = ("relu1_1", "relu2_1", "relu3_1", "relu4_1", "relu5_1")
style_weight = 1e4


def get_style_op(encoder, layer_weight):
    return GramOperator(encoder, score_weight=layer_weight)


style_loss = MultiLayerEncodingOperator(
    multi_layer_encoder, style_layers, get_style_op, score_weight=style_weight,
)

criterion = PerceptualLoss(content_loss, style_loss).to(device)
print(criterion)

Opposed to the prior examples we want to perform an NST on multiple resolutions. In
``pystiche`` this handled by an :class:`~pystiche.pyramid.ImagePyramid` . The
resolutions are selected by specifying the ``edge_sizes`` of the images on each level
. The optimization is performed for ``num_steps`` on the different levels.

The resizing of all images, i.e. ``input_image`` and target images (``content_image``
and ``style_image``) is handled by the ``pyramid``. For that we need to register the
perceptual loss (``criterion``) as ``resize_targets``.

<div class="alert alert-info"><h4>Note</h4><p>By default the ``edge_sizes`` correspond to the shorter ``edge`` of the images. To
  change that you can pass ``edge="long"``. For fine-grained control you can also
  pass a sequence comprising ``"short"`` and ``"long"`` to select the ``edge`` for
  each level separately.</p></div>



In [ ]:
edge_sizes = (500, 800)
num_steps = (500, 200)
pyramid = ImagePyramid(edge_sizes, num_steps, resize_targets=(criterion,))
print(pyramid)

Next up, we load and show the images that will be used in the NST.



In [ ]:
content_image = images["dancing"].read(device=device)
show_image(content_image, title="Input image")

In [ ]:
style_image = images["picasso"].read(device=device)
show_image(style_image, title="Output image")

<div class="alert alert-info"><h4>Note</h4><p>Although the images will be automatically resized during the optimization you might
  need to resize them before: if you are working with large source images you might
  run out of memory by setting up the targets of the perceptual loss. In that case it
  is good practice to resize the images upfront to the largest size the ``pyramid``
  will handle:

  .. code-block::

      top_level = pyramid[-1]
      image = top_level.resize(image)</p></div>



As a last preliminary step the previously loaded images are set as targets for the
perceptual loss (``criterion``) and we create the input image.



In [ ]:
criterion.set_content_image(content_image)
criterion.set_style_image(style_image)

starting_point = "content"
input_image = get_input_image(starting_point, content_image=content_image)
show_image(input_image, title="Input image")

Finally we run the NST and afterwards show the result and save it.



In [ ]:
output_image = default_image_pyramid_optim_loop(input_image, criterion, pyramid)

show_image(output_image, title="Output image")
write_image(output_image, "image_optimization_with_pyramid.jpg")